Imports

In [1]:
import pandas as pd
import numpy as np
from rdflib import Graph, URIRef, RDF, Literal
from rdflib.namespace import RDF, XSD
from copy import deepcopy

Retrieve DFs

In [2]:
moves = pd.read_pickle("../../cleanedDFs/moves.pkl")
house_prices = pd.read_pickle("../../cleanedDFs/prices.pkl")
house_availability = pd.read_pickle("../../cleanedDFs/availability.pkl")
population = pd.read_pickle("../../cleanedDFs/population.pkl")

Create townships dict to map the township code to the name of the township

In [3]:
gemeentes = pd.read_excel("../../Data/gemeentes.xlsx",header=None)
zip_iter = zip(gemeentes[0].to_list(), gemeentes[1].to_list())
municipalities_dict = dict(zip_iter)

Specify which columns to keep in the rdf for each table

In [4]:
relevant_cols_dict = {
    "Moves": ['TussenGemeentenVerhuisdePersonen_1'],
    "HousePrices": ['GemiddeldeWoningwaarde_4', 'Woningen_2'],
    "HouseAvailability": ['TotaleWoningvoorraad_1', 'StatusVanBewoning'],
    "Population": ["BevolkingOp31December_20"]
}

RDF Creation Functions

In [7]:
"""
Note to myself: I tried to create IRI's for the tablenames and 
columns/rows because IRI's are mentioned here:

https://www.w3.org/TR/rdb-direct-mapping/PR-to-REC#RDF-IRI
"""
def add_all_possible_years(rdf_graph):
    """
    Add all five years to the rdf graph.
    """
    for year in [np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]: 
        rdf_graph.add((URIRef(f"http://example.org/{year}"), RDF.type, XSD.gYear))
        
        #I think we should be able to get a more fitting property than an example.org property here
        rdf_graph.add((URIRef(f"http://example.org/{year}"), URIRef("http://example.org/hasValue"), Literal(year)))

    

def add_all_municipalities_to_rdf(rdf_graph, municipalities_dict):
    """
    For each municipality in the data, create a URIRef, and add to triple.
    E.g. for municipality GM0034, add triples

        (GM0034, rdf.type, municipality)
        'GM0034 is a municipality'
        &
        (GM0034, official_name, Almere)
        'GM0034's official name is Almere'
    
    to the rdf.
    """
    #Create nodes for the municipalities, will be the object of the triple
    municipality_URI = URIRef('http://rdfs.co/juso/Municipality')

    #name property of 'spatial things' -> e.g. a municipality
    official_name_URI = URIRef('http://rdfs.co/juso/official_name')

    #Get all unique municipalities. To be sure get the unique values from both 
    #RegioVanVestiging and RegioVanVestiging
    unique_municipalities = list(set(moves.RegioVanVertrek.unique().tolist() + moves.RegioVanVestiging.unique().tolist()))

    #loop over all municipalities, as we want to create triples for all municipalities
    for municipality in unique_municipalities:
        
        #this will be the subject of the triple
        municipality_node = URIRef(f"http://example.org/{municipality}")

        #get official name from the dict
        official_name_literal = Literal(municipalities_dict[municipality])

        #add the triples
        rdf_graph.add((municipality_node, RDF.type, municipality_URI))
        rdf_graph.add((municipality_node, official_name_URI, official_name_literal))


def from_table_to_rdf(rdf_graph, df, table_name):
    """
    Given a table, add all content from the table to the rdf.

    The resulting rdf will look as shown in rdf.pdf in the Images folder.
    """
    columns = df.columns.to_list()
    columns.remove('Perioden')
    
    #remove regio column(s), the triples relating to these columns 
    #will be added manually
    if 'RegioS' in columns:
            columns.remove('RegioS')
            single_regio_col = True
    else:
            columns.remove('RegioVanVestiging')
            columns.remove('RegioVanVertrek')
            single_regio_col = False
    
    #Based on slide 12 from lecture 7, the table name will be
    #the type of the common subjects
    table_IRI = URIRef(f"{table_name}")

    for _, row in df.iterrows():

        #create e.g. <Moves/ID=283955>
        row_IRI = URIRef(f"{table_name}/ID={row.name}")
        rdf_graph.add((row_IRI, RDF.type, table_IRI))

        #Link this row to the correct year
        rdf_graph.add((row_IRI, URIRef(f"{table_name}#Perioden"), URIRef(f"http://example.org/{row['Perioden']}"))) 

        #add triples corresponding to municipalities manually, because this is not as
        #simple as just adding a literal. We want to refer back to specific municipality node
        #as added to the rdf by add_all_municipalities_to_rdf()
        if single_regio_col:            
            municipality = row['RegioS']

            #E.g add (HouseAvailability/ID=664, HouseAvailability#RegioS, GM0034)
            rdf_graph.add((row_IRI, URIRef(f"{table_name}#RegioS"), URIRef(f"http://example.org/{municipality}")))

        else:
            municipality_vestiging = row['RegioVanVestiging']
            municipality_vertrek = row['RegioVanVertrek']

            # E.g. add (Moves/ID=283955, Moves#RegioVanVestiging, GM1680)
            # and      (Moves/ID=283955, Moves#RegioVanVertrek, GM0034)       
            rdf_graph.add((row_IRI, URIRef(f"{table_name}#RegioVanVestiging"), URIRef(f"http://example.org/{municipality_vestiging}")))
            rdf_graph.add((row_IRI, URIRef(f"{table_name}#RegioVanVertrek"), URIRef(f"http://example.org/{municipality_vertrek}")))

        #loop over leftover columns in order to add all other values
        for col in columns:
            col_IRI = URIRef(f"{table_name}#{col}")
            
            #If a value is NaN, don't put it in the rdf
            #nan is a float. If you dont do both check we get TypeErrors
            col_value = row[col]
            if isinstance(col_value, float):
                if np.isnan(col_value):
                    continue
            literal = Literal(col_value)

            rdf_graph.add((row_IRI, col_IRI, literal))
        
        

Actually creating the RDF

In [8]:
#Create rdf graph
g = Graph()

In [9]:
add_all_possible_years(g)

In [10]:
municipalities_dict['GM9999'] = 'UnknownMunicipality'

#add all township nodes to rdf
add_all_municipalities_to_rdf(g, municipalities_dict)

In [11]:
#add all info from tables to rdf
from_table_to_rdf(g, moves, "Moves")
from_table_to_rdf(g, house_prices, "HousePrices")
from_table_to_rdf(g, house_availability, "HouseAvailability")
from_table_to_rdf(g, population, "Population")

In [1]:
#Check results
# print(g.serialize(format='n3'))

Relevant variables/dict info for querying

In [13]:
#Get a clear idea of the relevant prefixes for the real deal
for prefix, namespace in g.namespaces():
       
    if namespace.startswith('Population'):
        population_prefix = prefix
    elif namespace.startswith('HouseAvailability'):
        availability_prefix = prefix
    elif namespace.startswith('HousePrices'):
        prices_prefix = prefix
    elif namespace.startswith('Moves'):
        moves_prefix = prefix
    elif namespace.startswith('http://example'):
        example_prefix = prefix


query_info_each_factor = {
    'population': {
        'table_prefix' : population_prefix,
        'table_name': 'Population',
        'var_of_interest': 'BevolkingOp31December_20'
    },
    'prices': {
        'table_prefix' : prices_prefix,
        'table_name': 'HousePrices',
        'var_of_interest': 'GemiddeldeWoningwaarde_4'
    },
    'availability_A': { 
        'table_prefix' : availability_prefix,
        'table_name': 'HouseAvailability',
        'var_of_interest': 'TotaleWoningvoorraad_1'
    }
}

Query helper functions

In [14]:
def select_clause_creator(yearless_variable_name, period, top_10_mun = "", take_average = True):
    """
    Return select clause.

    E.g. for yearless_variable_name = nmbr_moves,
             period = [2016,2017]
             take_average = True
    
    The result is:
        ( (?nmbr_moves2016+?nmbr_moves2017) / 2 AS ?nmbr_moves)
    
    If take_average = False, the result is:
        ( (?nmbr_moves2016+?nmbr_moves2017) AS ?nmbr_moves)
    """    
    string = "( ("

    for year in period:
        string += f'?{yearless_variable_name}_{top_10_mun}_{year}+' 

    string = string[:-1]
    if take_average:
        string += f') / {len(period)} AS ?{yearless_variable_name}_{top_10_mun})'
    else:
        string += f') AS ?{yearless_variable_name}_{top_10_mun})'

    return string

def where_clause_creator(factor, period, top_10_mun = ""):
        
    table_name = query_info_each_factor[factor]['table_name']
    table_prefix = query_info_each_factor[factor]['table_prefix']
    var_of_interest = query_info_each_factor[factor]['var_of_interest']
    
    string = ""

    for year in period:
        string += \
            f"""
            ?{factor}_instance_{top_10_mun+"_"}{year} a <{table_name}> .
            ?{factor}_instance_{top_10_mun+"_"}{year} {table_prefix}:RegioS {'?other_regio' if top_10_mun == "" else f'{example_prefix}:{top_10_mun}'} .
            ?{factor}_instance_{top_10_mun+"_"}{year} {table_prefix}:Perioden {example_prefix}:{year} .
            ?{factor}_instance_{top_10_mun+"_"}{year} {table_prefix}:{var_of_interest} ?{factor}_{top_10_mun}_{year} .
            """
                    
    return string

Query Function

In [15]:
def query_rdf(g, top_10_municipality, period, factor, top_10_is_vestiging = True, print_query = False, print_result = True):
    """Query the rdf graph to find all information about a given factor (population / availability_A / prices) for a 
    specific top 10 municipality and a specific time period"""

    select = f"""
    SELECT DISTINCT 
            ?other_regio
            {select_clause_creator('nmbr_moves', period, take_average = False)}
            {select_clause_creator(factor, period, top_10_mun=top_10_municipality)}
            {select_clause_creator(factor, period)}
    """

    where = """
        WHERE {"""

    for year in period:
        where += f"""
            ?moves_instance{year} a <Moves> .
            ?moves_instance{year} {moves_prefix}:RegioVanVestiging {f'{example_prefix}:{top_10_municipality}' if top_10_is_vestiging else '?other_regio'} .
            ?moves_instance{year} {moves_prefix}:RegioVanVertrek {f'{example_prefix}:{top_10_municipality}' if not top_10_is_vestiging else '?other_regio'} .
            ?moves_instance{year} {moves_prefix}:TussenGemeentenVerhuisdePersonen_1 ?nmbr_moves__{year} .
            ?moves_instance{year} {moves_prefix}:Perioden {example_prefix}:{year} .
            """

    where += where_clause_creator(factor,period,top_10_mun=top_10_municipality)
    where += where_clause_creator(factor,period)    

    where +=  \
        """}
        """

    order_by = """
            ORDER BY DESC(?nmbr_moves)
            """

    final_query = select + where + order_by

    if print_query:
        print(final_query)

    qres = g.query(final_query)
    
    if print_result:
        for row in qres:
            print(row)

    return qres

Run the query, and create + save DFS

In [16]:
#need to use factor availability_A, prices, population
def create_and_save_queried_df(rdf_graph, top_10_municipality,top_10_is_vestiging = True, save_df = False):
        """Run queries for all factors (prices, population, availability_A), and add results to a dataframe.
        The queries are run per factor and not with all factors at once as doing it per factor is faster. Furthermore,
        the visualisation asks for one factor at the time. 

        The resulting dataframe is saved in the QueryDFs folder. E.g. if this DF is about moves To Amsterdam,
        the DF is saved as 'To_Amsterdam.pkl'
        """
        
        df = pd.DataFrame(columns=['gemeente_code', 'gemeente_naam', 'year', 'moves', 'prices_other', 'prices_top_10', 'population_other', 'population_top_10', 'availability_other', 'availability_top_10'])

        #loop through the years to update
        for year in ['2016','2017','2018','2019','2020']:
        
                #per row in query, update df
                #eerst is dat dus alleen moves en price
                qres = query_rdf(rdf_graph,top_10_municipality,[year],'prices',top_10_is_vestiging,False,False)
                
                for row in qres:    
                        #to the df, add other_regio, name, year, #moves, price_other, price_top_10
                        other_regio = row['other_regio'][-6:]

                        df = df.append({'gemeente_code': other_regio,
                                                'gemeente_naam': municipalities_dict[other_regio],
                                                'moves': row.nmbr_moves_,
                                                'year': year,
                                                'prices_other': row.prices_,
                                                'prices_top_10': row[f'prices_{top_10_municipality}']},True)


                #run query for population
                #add to the df population_other_regio, population_top_10
                qres = query_rdf(rdf_graph,top_10_municipality,[year],'population',top_10_is_vestiging,False,False)

                for row in qres:
                        other_regio = row['other_regio'][-6:]
                        
                        index = df.loc[(df['gemeente_code'] == other_regio) & (df['year'] == year)].index
                        df.at[index, 'population_other'] = row.population_
                        df.at[index, 'population_top_10'] = row[f'population_{top_10_municipality}']


                # run query for availability
                # add to the df availavility_other_regio, availability_top_10
                qres = query_rdf(rdf_graph,top_10_municipality,[year],'availability_A',top_10_is_vestiging,False,False)

                for row in qres:
                        other_regio = row['other_regio'][-6:]
                        
                        index = df.loc[(df['gemeente_code'] == other_regio) & (df['year'] == year)].index
                        df.at[index, 'availability_other'] = row.availability_A_
                        df.at[index, 'availability_top_10'] = row[f'availability_A_{top_10_municipality}']

        if save_df:
                df.to_pickle(f"../../QueryDFs/{'To' if top_10_is_vestiging else 'From'}_{municipalities_dict[top_10_municipality]}.pkl")

        return df

In [18]:
#set to True if I want to save the dfs again
save_df = False

#add all possible queries to dfs and save
for municipality in ['GM0363','GM0599','GM0518','GM0344','GM0772','GM0855','GM0014','GM0034','GM0758','GM0268']:
    for top_10_is_vestiging in [True,False]:

        create_and_save_queried_df(g, municipality, top_10_is_vestiging, save_df)